## Teoretiska frågor


1. Lotta delar upp sin data i ”Träning”, ”Validering” och ”Test”, vad används respektive del för?

#### Svar: 
Träning: Används för att träna modellen, dvs. modellen lär sig från denna data, men det går inte att testa hur bra modellen  blivit på data som den redan känner till.
Validering: Används för att finjustera hyperparametrar och utvärdera modellen under träning för att undvika överträning.
Test: Data som inte finns med i träning eller valideringsdatan använder man för att slutligen utvärdera modellens prestanda.

2. Förklara (gärna med ett exempel): Ordinal encoding, one-hot encoding, dummy variable encoding.

#### Svar: 
Ordinal encoding:   Tilldelar varje kategori ett numeriskt värde baserat på en naturlig ordning.   
Exempel: Liten (1), Medel (2), Stor (3)  

One-hot encoding: Skapar en binär kolumn för varje kategori, där endast en kolumn har värdet 1 per observation.  
Exempel:  
Hund → [1, 0, 0]  
Katt → [0, 1, 0]  
Fågel → [0, 0, 1]  

Dummy variable encoding: Liknar one-hot encoding men tar bort en kategori för att undvika multikollinearitet, dvs man vill få bort starka samband i de oberoende variablerna. Är de korrelerande kan det bli svårt för en modell att veta vilken variabel som påverkar mest.  
Exempel: Om vi har tre kategorier (A, B, C), kodas de som:  
A → [0, 0]  
B → [1, 0]  
C → [0, 1]  

3. Göran påstår att datan antingen är ”ordinal” eller ”nominal”. Julia säger att detta måste tolkas. Hon ger ett exempel med att färger såsom {röd, grön, blå} generellt sett inte har någon inbördes ordning (nominal) men om du har en röd skjorta så är du vackrast på festen (ordinal) – vem har rätt?

#### Svar: 
Båda har väl rätt på sitt sätt: Datan är antingen nominal eller ordinal, men nominal data kan bli ordinal om man lägger till värdering till något som inte har det i vanliga fall.

Nominal data har ingen inbördes ordning (ex. färger, djurarter, nationaliteter).
Ordinal data har en logisk ordning (ex. betygsskala, klädstorlekar).

Men tolkar man att man får högre rang av en röd skjorta, så har man infört en rangordning och färgen blir ordinal i det sammanhanget.

4. Vad används joblib och pickle till?

## Svar:
Det är två bibliotek som används för att lagra stora python-objekt.   
Pickle verkar rekommenderas för något mindre objekt, medan joblib verkar bra på att hantera större objekt som NumPy-arrayer, ML-modeller och scikit-modeller.

## Modellera MNIST
Använd maskininlärning för att modellera MNIST datan. Du skall utvärdera minst två olika modeller i ditt arbete och göra ett komplett ML-flöde, från början, där du laddar in data, till slut, där du utvärderar den bäst valda modellen på din test data. Hur du laddar ned MNIST datan kan du se här.

Jag har valt att testa tre olika modeller som jag tror kommer fungera ganska bra med tanke på vad vi lärt oss i kursen och som även ingått i kursen, (samt att jag haft lite dåligt med tid sista två veckorna så för min egen skull vill jag också testa fler):  
Support Vector Machine (SVM) och Random Forest (RF) och Extra Tree (ET) 

(Först har jag installerat joblib).  
Börja med alla importer och installationer:

In [1]:
import numpy as np
import joblib
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Jag laddar ner MNIST-datasetet:

In [2]:
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
X = mnist["data"][:70000]
y = mnist["target"].astype(np.uint8)[:70000]

Från datasetets beskrivning vet vi att vi kan använda de första 70.000 värdena för att träna på machinelearning, då de bearbetat den datan för att vara bra träningsdata. Så jag delar in den i träning, test och valideringsdata. 50000 i test och 10000 i test och validering respektive.

In [3]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=50000, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=10000, random_state=42)

Skala data för SVM (ET och RF behöver inte skalning):

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

Jag använder gridsearch för att optimera hyperparametrar och för att slippa repetativ kod så skapar jag en funktion 

In [5]:
def optimaze_train_and_evaluate_model(model, param_grid, X_train, y_train, X_val, y_val, scaled=False):
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_scaled if scaled else X_train, y_train)
    best_model = grid_search.best_estimator_
    accuracy = accuracy_score(y_val, best_model.predict(X_val_scaled if scaled else X_val))
    return best_model, accuracy

Välj hyperparametrar och skicka in i vår funktion:

In [6]:
extra_trees_params = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}
extra_trees, et_acc = optimaze_train_and_evaluate_model(ExtraTreesClassifier(random_state=42), extra_trees_params, X_train, y_train, X_val, y_val)

random_forest_params = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}
random_forest, rf_acc = optimaze_train_and_evaluate_model(RandomForestClassifier(random_state=42), random_forest_params, X_train, y_train, X_val, y_val)

svm_params = {'C': [1, 10, 100], 'gamma': ['scale', 'auto']}
svm, svm_acc = optimaze_train_and_evaluate_model(SVC(kernel='rbf'), svm_params, X_train_scaled, y_train, X_val_scaled, y_val, scaled=True)

Jag väljer bäst modell enligt valideringsdata:

In [7]:
model_scores = {
    "Extra Trees": et_acc,
    "Random Forest": rf_acc,
    "SVM": svm_acc
}

best_model_name = max(model_scores, key=model_scores.get)
best_model = {"Extra Trees": extra_trees, "Random Forest": random_forest, "SVM": svm}[best_model_name]
print(f"Bästa modellen baserat på valideringsdata: {best_model_name} med accuracy: {model_scores[best_model_name]}")


Bästa modellen baserat på valideringsdata: SVM med accuracy: 0.9712


Sen evaluerar jag den bästa modellen med testdatan:

In [8]:
best_test_acc = accuracy_score(y_test, best_model.predict(X_test if best_model_name != "SVM" else X_test_scaled))
print(f"Bästa modellens test-accuracy: {best_test_acc}")

Bästa modellens test-accuracy: 0.968


Sen sparar jag ner den bästa modellen i förhoppning om att jag hinner gör VG-delen, för den verkar väldigt rolig!

In [9]:
joblib.dump(best_model, "best_mnist_model.joblib")

['best_mnist_model.joblib']